In [1]:
import pandas as pd
from sklearn.model_selection import KFold
import re
import os
import shutil
import random
import json

## output.txtからcsvに書き換える

In [167]:
import re
import json
import pandas as pd


num_kf = ["0", "1", "2", "3", "4"]
name_respondent = ["a-y", "y-k", "h-r", "k-y", "t-h"]

# num_kf = ["0"]
# name_respondent = ["a-y"]

for name in name_respondent:
    for kf in num_kf:
        # Open the text file and read its contents
        path_name = f'/home/taki/Otter/log/one_input_{name}kf={kf}_batch16_lr=1e-4_warmup=0.5_epoch=50/test/query_only/output'
        with open(f'{path_name}.txt', 'r') as file:
            text = file.read()

        entries = re.split(r'[-]{40,}', text.strip())
        key_list = []
        gpt_list = []
        answer_list = []

        for entry in entries:
            lines = entry.strip().split('\n')
            # print(lines)
            if len(lines) >= 3:  # linesに少なくとも4つの要素が必要です
                key = lines[0]
                key_list.append(key)
                gpt_value = lines[1].split('GPT:')[1].strip()#.replace("Yes", "").replace("No", "").replace(",", "",1)
                answer_value = lines[2].split('Answer:')[1].strip()#.replace("Yes", "").replace("No", "").replace(",", "",1)
                gpt_list.append(gpt_value)
                answer_list.append(answer_value)

                
        # Convert the matches to a DataFrame
        
        df = pd.DataFrame({
            'Query': key_list,
            'GPT': gpt_list,
            'Answer': answer_list
        })

        df.to_csv(f'/home/taki/Otter/log/for_gpt_and_human/output_{name}_kf={kf}_batch16_lr=1e-4_warmup=0.5_epoch=50.csv', index=False)


## human評価用のcsvを作成。各交差検証の結果から10枚ずつサンプリング

In [172]:
num_kf = ["0", "1", "2", "3", "4"]
name_respondent = ["a-y", "y-k", "h-r", "k-y", "t-h"]

df_new = pd.DataFrame()  # Create an empty DataFrame

for name in name_respondent:
    for kf in num_kf:
        path_name = f'/home/taki/Otter/log/for_gpt_and_human/all_data/output_{name}_kf={kf}_batch16_lr=1e-4_warmup=0.5_epoch=50.csv'
        df = pd.read_csv(path_name)
        
        div = int(kf) % 5 *10
        df_temp = df.iloc[div:div+10, :]

        # Add a new column 'name_kf'
        df_temp['name_kf'] = f'{name}_kf={kf}'
        
        df_new = pd.concat([df_new, df_temp])
# df_newからYesとNoを削除
df_new = df_new.replace('Yes, ', '', regex=True)
df_new = df_new.replace('No, ', '', regex=True)

df_new.to_csv(f'/home/taki/Otter/log/for_gpt_and_human/for_eval_data.csv', index=False)



/tmp/ipykernel_2200414/3150882970.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['name_kf'] = f'{name}_kf={kf}'
/tmp/ipykernel_2200414/3150882970.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['name_kf'] = f'{name}_kf={kf}'
/tmp/ipykernel_2200414/3150882970.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [152]:
df.iloc[0]

Query                                    egg+egg+above00500
GPT        Because there are bubbles in the white, makin...
Answer     Because there are bubbles in the white, but t...
Name: 0, dtype: object

In [162]:
for i in range(5):
    div = int(i) % 5
    print(div*10)

0
10
20
30
40
